In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import minmax_scale

# Get weather dataset

In [23]:
weather_dataset = pd.read_csv('weather_wwo_manila_2015.csv', skipinitialspace=True, encoding='cp1252')
cols = ["tempC", "windspeedKmph", "cond", "precipMM", "humidity", "visibility", "pressure", "cloudcover", "heatIndexC", "dewPointC", "windChillC", "windGustKmph", "feelsLikeC"]
date_dataset = weather_dataset["dt"]
weather_dataset = weather_dataset[cols]

# Correlate traffic and weather dataset

In [ ]:
roads = ["A. Maceda", "Anda Circle", "Antipolo", "Bluementritt", "Buendia", "Edsa Extension", "Finance Road", "Gov. Forbes - Lacson", "Lerma", "Magsaysay Ave", "P.Noval", "Pablo Ocampo", "Pedro Gil", "Quezon Ave.", "Quirino", "Rajah Sulayman", "Taft Ave.", "U.N. Avenue", "Vicente Cruz"]

In [ ]:
for road in roads:
    traffic_dataset = pd.read_csv('mmda_2015_transformed/mmda_' + road + '_2015_transformed.csv', skipinitialspace=True, encoding='cp1252')
    traffic_dataset = traffic_dataset[['statusN', 'statusS']]
    
    # merge traffic and weather dataset
    dataset = pd.concat([traffic_dataset, weather_dataset], axis=1, join='inner')
    
    # correlate
    corr = dataset.corr(method='spearman')
    corr.to_csv('corr_mmda_wwo_2015/corr_results_mmda_wwo_' + road + '_2015.csv')

# Correlate traffic and weather dataset wth lags

In [3]:
roads = ["A. Maceda", "Anda Circle", "Antipolo", "Bluementritt", "Buendia", "Edsa Extension", "Finance Road", "Gov. Forbes - Lacson", "Lerma", "Magsaysay Ave", "P.Noval", "Pablo Ocampo", "Pedro Gil", "Quezon Ave.", "Quirino", "Rajah Sulayman", "Taft Ave.", "U.N. Avenue", "Vicente Cruz"]

In [6]:
total_lags = 8

for road in roads:
    traffic_dataset = pd.read_csv('mmda_2015_transformed/mmda_' + road + '_2015_transformed.csv', skipinitialspace=True, encoding='cp1252')
    traffic_dataset = traffic_dataset[['statusN', 'statusS']]
    
    # merge traffic and weather dataset
    dataset = pd.concat([traffic_dataset, weather_dataset], axis=1, join='inner')
    
    # make a copy
    new_dataset = dataset.copy()
    
    for i in range(1, (total_lags+1)):
        new_dataset.statusN = new_dataset.statusN.shift(-1)
        new_dataset.statusS = new_dataset.statusS.shift(-1)
        new_dataset = new_dataset[:(len(new_dataset)-1)]

        corr = new_dataset.corr(method='spearman')
        corr.to_csv('corr_mmda_wwo_2015_lags/corr_mmda_wwo_' + road + '_2015_lag_' + str(i) + '.csv')

In [22]:

for i in range(1, (total_lags+1)):
    
    lag_dataset = []
    
    for road in roads:
        dataset = pd.read_csv('corr_mmda_wwo_2015_lags/corr_mmda_wwo_' + road + '_2015_lag_' + str(i) + '.csv', skipinitialspace=True, encoding='cp1252')
        dataset = dataset.loc[:, ~dataset.columns.str.contains('^Unnamed')]
        dataset = dataset.loc[:1]
        
        lag_dataset.append(dataset)
    
    lag_dataset = pd.concat(lag_dataset)
    lag_dataset.to_csv('corr_mmda_wwo_2015_lags/corr_mmda_wwo_2015_lag_' + str(i) + '.csv')

# Correlate traffic and weather with seasons

In [24]:
date_dataset

0           1/1/2015 0:00
1           1/1/2015 0:15
2           1/1/2015 0:30
3           1/1/2015 0:45
4           1/1/2015 1:00
5           1/1/2015 1:15
6           1/1/2015 1:30
7           1/1/2015 1:45
8           1/1/2015 2:00
9           1/1/2015 2:15
10          1/1/2015 2:30
11          1/1/2015 2:45
12          1/1/2015 3:00
13          1/1/2015 3:15
14          1/1/2015 3:30
15          1/1/2015 3:45
16          1/1/2015 4:00
17          1/1/2015 4:15
18          1/1/2015 4:30
19          1/1/2015 4:45
20          1/1/2015 5:00
21          1/1/2015 5:15
22          1/1/2015 5:30
23          1/1/2015 5:45
24          1/1/2015 6:00
25          1/1/2015 6:15
26          1/1/2015 6:30
27          1/1/2015 6:45
28          1/1/2015 7:00
29          1/1/2015 7:15
               ...       
35010    12/31/2015 16:30
35011    12/31/2015 16:45
35012    12/31/2015 17:00
35013    12/31/2015 17:15
35014    12/31/2015 17:30
35015    12/31/2015 17:45
35016    12/31/2015 18:00
35017    12/